In [1]:
import torch
from torchvision.models import resnet18

In [5]:
model = resnet18()
model.eval()
print()

## 模型转换

In [8]:
dummy_input = torch.randn(1, 3, 224, 224)

In [9]:
with torch.no_grad():
    jit_model = torch.jit.trace(model, dummy_input)

In [10]:
jit_model

ResNet(
  original_name=ResNet
  (conv1): Conv2d(original_name=Conv2d)
  (bn1): BatchNorm2d(original_name=BatchNorm2d)
  (relu): ReLU(original_name=ReLU)
  (maxpool): MaxPool2d(original_name=MaxPool2d)
  (layer1): Sequential(
    original_name=Sequential
    (0): BasicBlock(
      original_name=BasicBlock
      (conv1): Conv2d(original_name=Conv2d)
      (bn1): BatchNorm2d(original_name=BatchNorm2d)
      (relu): ReLU(original_name=ReLU)
      (conv2): Conv2d(original_name=Conv2d)
      (bn2): BatchNorm2d(original_name=BatchNorm2d)
    )
    (1): BasicBlock(
      original_name=BasicBlock
      (conv1): Conv2d(original_name=Conv2d)
      (bn1): BatchNorm2d(original_name=BatchNorm2d)
      (relu): ReLU(original_name=ReLU)
      (conv2): Conv2d(original_name=Conv2d)
      (bn2): BatchNorm2d(original_name=BatchNorm2d)
    )
  )
  (layer2): Sequential(
    original_name=Sequential
    (0): BasicBlock(
      original_name=BasicBlock
      (conv1): Conv2d(original_name=Conv2d)
      (bn1): B

In [16]:
jit_layer1 = jit_model.layer1
print(jit_layer1)

Sequential(
  original_name=Sequential
  (0): BasicBlock(
    original_name=BasicBlock
    (conv1): Conv2d(original_name=Conv2d)
    (bn1): BatchNorm2d(original_name=BatchNorm2d)
    (relu): ReLU(original_name=ReLU)
    (conv2): Conv2d(original_name=Conv2d)
    (bn2): BatchNorm2d(original_name=BatchNorm2d)
  )
  (1): BasicBlock(
    original_name=BasicBlock
    (conv1): Conv2d(original_name=Conv2d)
    (bn1): BatchNorm2d(original_name=BatchNorm2d)
    (relu): ReLU(original_name=ReLU)
    (conv2): Conv2d(original_name=Conv2d)
    (bn2): BatchNorm2d(original_name=BatchNorm2d)
  )
)


In [23]:
jit_layer1.graph

graph(%self.11 : __torch__.torch.nn.modules.container.Sequential,
      %4 : Float(1, 64, 56, 56, strides=[200704, 3136, 56, 1], requires_grad=0, device=cpu)):
  %_1.1 : __torch__.torchvision.models.resnet.___torch_mangle_10.BasicBlock = prim::GetAttr[name="1"](%self.11)
  %_0.1 : __torch__.torchvision.models.resnet.BasicBlock = prim::GetAttr[name="0"](%self.11)
  %6 : Tensor = prim::CallMethod[name="forward"](%_0.1, %4)
  %7 : Tensor = prim::CallMethod[name="forward"](%_1.1, %6)
  return (%7)

## 模型优化 

In [25]:
# 调用inline pass，对graph做变换 
torch._C._jit_pass_inline(jit_layer1.graph) 
print(jit_layer1.code) 

"""
上面代码中我们使用了一个名为inline的pass，将所有子模块进行内联，这样我们就能看见更完整的推理代码。
pass是一个来源于编译原理的概念，一个 TorchScript 的 pass 会接收一个图，遍历图中所有元素进行某种变换，
生成一个新的图。我们这里用到的inline起到的作用就是将模块调用展开，尽管这样做并不能直接影响执行效率，
但是它其实是很多其他pass的基础。PyTorch 中定义了非常多的 pass 来解决各种优化任务，
未来我们会做一些更详细的介绍。 
"""

def forward(self,
    argument_1: Tensor) -> Tensor:
  _1 = getattr(self, "1")
  _0 = getattr(self, "0")
  bn2 = _0.bn2
  conv2 = _0.conv2
  relu = _0.relu
  bn1 = _0.bn1
  conv1 = _0.conv1
  weight = conv1.weight
  input = torch._convolution(argument_1, weight, None, [1, 1], [1, 1], [1, 1], False, [0, 0], 1, False, False, True, True)
  running_var = bn1.running_var
  running_mean = bn1.running_mean
  bias = bn1.bias
  weight0 = bn1.weight
  input0 = torch.batch_norm(input, weight0, bias, running_mean, running_var, False, 0.10000000000000001, 1.0000000000000001e-05, True)
  input1 = torch.relu_(input0)
  weight1 = conv2.weight
  input2 = torch._convolution(input1, weight1, None, [1, 1], [1, 1], [1, 1], False, [0, 0], 1, False, False, True, True)
  running_var0 = bn2.running_var
  running_mean0 = bn2.running_mean
  bias0 = bn2.bias
  weight2 = bn2.weight
  out = torch.batch_norm(input2, weight2, bias0, running_mean0, running_var0, False, 0.10000000000000001, 1.0000000000000001e-05, True

'\n上面代码中我们使用了一个名为inline的pass，将所有子模块进行内联，这样我们就能看见更完整的推理代码。\npass是一个来源于编译原理的概念，一个 TorchScript 的 pass 会接收一个图，遍历图中所有元素进行某种变换，\n生成一个新的图。我们这里用到的inline起到的作用就是将模块调用展开，尽管这样做并不能直接影响执行效率，\n但是它其实是很多其他pass的基础。PyTorch 中定义了非常多的 pass 来解决各种优化任务，\n未来我们会做一些更详细的介绍。 \n'

## 序列化

In [26]:
# 将模型序列化 
jit_model.save('jit_model.pth')
# 加载序列化后的模型 
jit_model = torch.jit.load('jit_model.pth') 

序列化后的模型不再与 python 相关，可以被部署到各种平台上。

PyTorch 提供了可以用于 TorchScript 模型推理的 c++ API，序列化后的模型终于可以不依赖 python 进行推理了： 

### 与onnx关系
ONNX 是业界广泛使用的一种神经网络中间表示，PyTorch 自然也对 ONNX 提供了支持。
torch.onnx.export函数可以帮助我们把 PyTorch 模型转换成 ONNX 模型，
这个函数会使用 trace 的方式记录 PyTorch 的推理过程。聪明的同学可能已经想到了，没错，ONNX 的导出，使用的正是 TorchScript 的 trace 工具。具体步骤如下：

- 使用 trace 的方式先生成一个 TorchScipt 模型，如果你转换的本身就是 TorchScript 模型，则可以跳过这一步。 
- 使用许多 pass 对 1 中生成的模型进行变换，其中对 ONNX 导出最重要的一个 pass 就是ToONNX，这个 pass 会进行一个映射，将 TorchScript 中prim、aten空间下的算子映射到onnx空间下的算子。 
- 使用 ONNX 的 proto 格式对模型进行序列化，完成 ONNX 的导出。 

### 与torch.fx关系
PyTorch1.9 开始添加了torch.fx工具，根据官方的介绍，它由符号追踪器（symbolic tracer），中间表示（IR）， 
Python 代码生成（Python code generation）等组件组成，实现了python->python的翻译。是不是和 TorchScript 看起来有点像？ 
其实他们之间联系不大，可以算是互相垂直的两个工具，为解决两个不同的任务而诞生。
- TorchScript 的主要用途是进行模型部署，需要记录生成一个便于推理优化的 IR，对计算图的编辑通常都是面向性能提升等等，不会给模型本身添加新的功能。 
- FX 的主要用途是进行python->python的翻译，它的 IR 中节点类型更简单，比如函数调用、属性提取等等，这样的 IR 学习成本更低更容易编辑。使用 FX 来编辑图通常是为了实现某种特定功能，比如给模型插入量化节点等，避免手动编辑网络造成的重复劳动。 

这两个工具可以同时使用，比如使用 FX 工具编辑模型来让训练更便利、功能更强大；然后用 TorchScript 将模型加速部署到特定平台。 